# Day 2: Exploring tabulated data with Pandas

Pandas is the best-in-slot tabulated data package in Python. Due to its immense popularity there is a vast variety of functionality build into it and a wealth of resources available on working with Pandas online. To start things off we will import pandas and give it the nickname 'pd' (Once again a convention within the Python community).

In [ ]:
# We import pandas and abbreviate it to pd
import pandas as pd
# We are also going to need matplotlib to plot our data
import matplotlib.pyplot as plt

So first let us open our dear friend: the Microsoft Excel file.

In [ ]:
# We can point to an Excel file by providing the path to the file
excel_file = pd.ExcelFile("./data/Tabulated/CTS_Database.xlsx")

There is a few things to note from our previous cell of code:
- First: We declare the Excel file like we would a variable. We need an object to contain the data of the Excel file.
- Secondly: We use the ExcelFile method of pandas to open Excel Files.
- Third: We use the file path relative to the script to locate the file. 

We can see which sheets are in the Excel file by calling the sheet_names property of Excel File

In [ ]:
# And see the name of each sheet by
excel_file.sheet_names

Let's inspect the Time Waterlevel sheet. A sheet can of course contain a lot of data. So if we just want to view the top few records of a sheet we do so with the .head() method.
- You can also specify a number within the parenthesis of the method to show that many record (e.g. .head(10) to show the first 10 records.)

In [ ]:
# We can open a specific sheet as follows
df_wl = excel_file.parse("Time WL")
# And take a peak at the files contents
df_wl.head()

For some more information regarding the number of records and the different types of columns present in the sheet we can use the .info() method:

In [ ]:
# We can get some information
df_wl.info()

From the above we can see that there are four columns in the sheet, with a total of 690 records. The .info() method will also tell us the data type of each column.

Now let us get a list of the unique SiteNames present in the sheet:

In [ ]:
print(df_wl["SiteName"].unique())

From the above code we can see that columns are indexed the same way we would a list or dictionary, except instead of an index we provide the name of the column. Then once we have the data for just that column we then proceed to call the .unique() method to get all unique values in that column.

Using the above list, let's take a look at the last unique SiteName: NH3. In order to get all the records matching the SiteName NH3 we will use the .loc method of the dataframe. When using the .loc method we do provide parenthesis, but instead provide square brackets to indicate the range we are interested in. Then within the square brackets we use a statement that effectively says:

From our dataframe, get me each record whose SiteName column is equal to NH3.

In [ ]:
# If we only want to look at one borehole's data
df_NH3 = df_wl.loc[df_wl["SiteName"] == "NH3"]
df_NH3.head()

Since we are now looking at only NH3 in the new df_NH3 dataframe, we don't need that SiteName column anymore. So let us make the DateTimeMeas our new index column and get rid of the NH3 column. While we are at it we will also delete the Status column which doesn't contain any records.

In [ ]:
# Let's make the DateTimeMeas the index of the table
df_NH3.set_index("DateTimeMeas", inplace=True)
# And drop the status and sitename columns
df_NH3 = df_NH3.drop(columns=["SiteName", "Status"])
df_NH3.head()

Now that we just have the DateTimeMeas and Waterlevel columns for NH3, let's get some basic statistics for the data using the .describe() method of the dataframe:

In [ ]:
# Get a description of our data
df_NH3.describe()

We can use these basic statistics to determine if any outliers are present and if we need to modify the data. In this case, we can see that we have 278 waterlevels, with an average waterlevel of approximately 12.6 meters below ground level. Both the maximum and minimum values appear valid. 

Now the best way to further explore the data would be to first plot a graph of the data. Pandas provides build-in methods to automatically plot data for us with matplotlib, let's take a look at the code:

In [ ]:
# Plot the data
df_NH3.plot(
    kind="line",
    color="red",
    xlabel="Date",
    ylabel="Water level (???)",
    title="Plot of NH3 water level"
)
# Show the figure
plt.show()

The code to plot the figure looks a bit different when we use Pandas, let's break it down:
- We call the .plot() method on the dataframe to plot it.
- Additionally, we can define some parameters to customise the plot
- With the "kind" property we define what kind of graph we would like.
- The "color" property is self explanatory.
- "xlabel" and "ylabel" is used to give title to the x- and y-axis respectively.
- The "title" propoerty is also self explanatory.

And voila, we have a graph. However, it is clear that something is wrong here. This data appears to be measuring the waterlevel from a certain datum located below the water level. In reality, this is data freshly extracted from a pressure transducer which has not been converted to meters below ground level yet. Let us correct that, the water level measured in NH3 when the pressure transducer was removed (the final reading) is 15.5 mbgl. Let's convert the data to the new datum:

In [ ]:
# First, let us get the last water level recording from the data
last_record = df_NH3["Waterlevel"].iloc[-1]
print(last_record)

The process was similar to what we used before. First, we only grab the Waterlevel column from the dataset and then we get the last position by using the index locator (.iloc) method and the same indexing style we would use with a list. With this we get a water level of 10.40 meters above datum, which is then equivalent to our 15.5 mbgl measurement. This puts the installation depth of our logger at:

15.5 + 10.4 = 25.9 mbgl

Now we can change our data to mbgl by subtracting the recorded value from this number:

In [ ]:
df_NH3["CorrWaterlevel"] = 25.9 - df_NH3["Waterlevel"]
df_NH3.describe()

Now we can plot our data again, but since our dataframe has two data columns now we will specify that it needs to only plot the corrected waterlevel column:

In [ ]:
# Plot the data
df_NH3["CorrWaterlevel"].plot(
    kind="line",
    color="blue",
    xlabel="Date",
    ylabel="Water level (meters below ground level)",
    title="Plot of NH3 water level"
)
# Since we are now working with mbgl, it would help visualisation to invert the y-axis
plt.gca().invert_yaxis()
# Show the figure
plt.show()

And that is exactly what we expected to see. Let us now move on to some common parts of data exploration and preparation. Anyone that has worked with real data knows how messy it can be. We will now explore some common workflows for handling data.

# Data Quality Checks and Fixes

First, let us open the BasicInfo sheet of the WISH database.

In [ ]:
# We can open the basic info sheet as follows
df_basic = excel_file.parse("BasicInf")
# And take a peak at the files contents
df_basic.head()

Now, let us look at how to handle a common problem: duplicates in our data.

In [ ]:
complete_duplicates = df_basic[df_basic.duplicated() == True]
complete_duplicates

We can remove the complete duplicates by calling the .drop_duplicates() method of the dataframe and passing no arguments.

In [ ]:
# The drop duplicates method will drop all records that are complete duplicates of another one
df_basic = df_basic.drop_duplicates()

If we want to find all duplicates just based on a specific column then we specify that column. The "keep" property is usually true by default. Here we set it to False so that it shows us the duplicates and the first instance of the same name:

In [ ]:
duplicate_names = df_basic[df_basic.duplicated('SiteName', keep=False) == True]
duplicate_names

It's clear that the duplicates which are the second instance of the name has the same values, just at a lower accuracy than the first instances. So we will drop the second instances using the .drop_duplicates() method and passing the column that we would like to be checked. Note that 'keep' property is now kept at its default value, which will keep the first instance of a SiteName but remove and duplicates following it.

In [ ]:
df_basic = df_basic.drop_duplicates('SiteName')

Now, let's look at some information regarding our dataframe:

In [ ]:
df_basic.info()

Next, let's drop the records where Xcoord or Ycoord is missing. But first, we'll make a dataframe listing those values so that we can send it to our client for feedback.

In [ ]:
df_missing_coords = df_basic[df_basic['Xcoord'].isna() | df_basic['Ycoord'].isna()]
df_missing_coords.to_excel('.\output\missing_coordinates.xlsx')  # Show how to fix the warning
df_missing_coords

In [ ]:
df_basic = df_basic.dropna(subset=['Xcoord', 'Ycoord'])
df_basic.info()

That problem is solved, let's look at some statistics of the dataaset:

In [ ]:
df_basic.describe()

From the statistics we can see we have some coordinates which have been swapped around. Let's fix that:

In [ ]:
swapped = df_basic[df_basic['Xcoord'] < -1000000]  # assuming Y will be a larger negative value than X
swapped

Now that we know which two records have the problems, we can swap their coordinates around:

In [ ]:
x_coord_UO21 = df_basic.loc[df_basic['SiteName'] == 'UO21']['Ycoord'].iloc[0]
y_coord_UO21 = df_basic.loc[df_basic['SiteName'] == 'UO21']['Xcoord'].iloc[0]

df_basic.loc[df_basic['SiteName'] == 'UO21', 'Xcoord'] = x_coord_UO21
df_basic.loc[df_basic['SiteName'] == 'UO21', 'Ycoord'] = y_coord_UO21

x_coord_UP15 = df_basic.loc[df_basic['SiteName'] == 'UP15']['Ycoord'].iloc[0]
y_coord_UP15 = df_basic.loc[df_basic['SiteName'] == 'UP15']['Xcoord'].iloc[0]

df_basic.loc[df_basic['SiteName'] == 'UP15', 'Xcoord'] = x_coord_UP15
df_basic.loc[df_basic['SiteName'] == 'UP15', 'Ycoord'] = y_coord_UP15

df_basic.describe()

We can see we also have some problems in the Depth column. Let's look for unrealistic depth values and put them in a dataframe to send to our client:

In [ ]:
df_invalid_depth = df_basic[(df_basic['Depth'] < 0) | (df_basic['Depth'] > 500)]
df_invalid_depth.to_excel(r'.\output\invalid_depth.xlsx')
df_invalid_depth

And that concludes our examples for DataFrames and Pandas. Now let's start practicing:

# DataFrame Exercise:

Prepare and clean the borehole database provided in the file "borehole_data.csv". A few of the problems we discussed today is present in the database. With the help of the examples above prepare the database for use. The file is loaded for you in the cell below. Add as many cells as you wish to complete this exercise.

In [ ]:
# First we open the file into a pandas dataframe
df_csv = pd.read_csv("./data/Tabulated/borehole_data.csv")
df_csv.head()

In [ ]:
# Start from here